In [1]:
import pandas as pd
import numpy as np
import json
import os
import glob
from tqdm import tqdm

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
path = '/home/manoelflorencio/cta_for_jd/testbedXS'
os.chdir(path)
print(os.getcwd())

/home/manoelflorencio/cta_for_jd/testbedXS


In [4]:
def generate_key_for_evaluation(df):
    left_table_name = []
    left_column_name = []
    right_table_name = []
    right_column_name = []
    df_key = pd.DataFrame((), columns=df.columns)
    
    for i in range(df.shape[0]):
        rtn = df.iloc[i, 2]
        ltn = df.iloc[i, 0]
        right_table_name.append(rtn if rtn > ltn else ltn)
        left_table_name.append(rtn if rtn < ltn else ltn)
    
        rcn = df.iloc[i, 3]
        lcn = df.iloc[i, 1]
        right_column_name.append(rcn if rtn > ltn else lcn)
        left_column_name.append(rcn if rtn < ltn else lcn)
    
    d = {'LEFT_TABLE':left_table_name,
         'LEFT_COLUMN':left_column_name,
         'RIGHT_TABLE':right_table_name,
         'RIGHT_COLUMN':right_column_name}
    df_key = pd.DataFrame(d)
    df_key = df_key.drop_duplicates()
    df_key = df_key.reset_index(drop=True)
    
    key = []
    for i in range(df_key.shape[0]):
        key.append("#".join(df_key.iloc[i,:]))
    df_key['KEY'] = key

    return df_key

In [5]:
def fix_table_names(table_name):
    return table_name + '.csv'

In [6]:
cta_matches = pd.read_csv('Description_test/CTA_from_descriptions/junio_JD_from_cta_matches.csv')
cta_matches = cta_matches[cta_matches['JOINABLE'] == 'Yes'][['LEFT_TABLE','LEFT_COLUMN','RIGHT_TABLE','RIGHT_COLUMN']]
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN
2,eo_pr,EIN,eo_xx,EIN
24,eo4,EIN,eo_xx,EIN
34,eo_xx,NAME,schools,SCHOOL_NAME
35,eo_pr,NAME,eo_xx,NAME
36,eo_xx,NAME,rental-standards-current-issues,BUSINESSOPERATOR


In [7]:
cta_matches['LEFT_TABLE'] = fix_table_names(cta_matches['LEFT_TABLE'])
cta_matches['RIGHT_TABLE'] = fix_table_names(cta_matches['RIGHT_TABLE'])

In [8]:
cta_matches = generate_key_for_evaluation(cta_matches)
cta_matches.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,eo_pr.csv,EIN,eo_xx.csv,EIN,eo_pr.csv#EIN#eo_xx.csv#EIN
1,eo4.csv,EIN,eo_xx.csv,EIN,eo4.csv#EIN#eo_xx.csv#EIN
2,eo_xx.csv,NAME,schools.csv,SCHOOL_NAME,eo_xx.csv#NAME#schools.csv#SCHOOL_NAME
3,eo_pr.csv,NAME,eo_xx.csv,NAME,eo_pr.csv#NAME#eo_xx.csv#NAME
4,eo_xx.csv,NAME,rental-standards-current-issues.csv,BUSINESSOPERATOR,eo_xx.csv#NAME#rental-standards-current-issues...


In [7]:
cta_matches.shape

(189, 5)

In [9]:
groundTruth = pd.read_csv('joinable_columns_gt3_quality.csv')
groundTruth = groundTruth[['ds_name', 'att_name', 'ds_name_2', 'att_name_2']]
groundTruth = generate_key_for_evaluation(groundTruth)
groundTruth.head()

,LEFT_TABLE,LEFT_COLUMN,RIGHT_TABLE,RIGHT_COLUMN,KEY
0,road-ahead-current-road-closures.csv,COMP_DATE,road-ahead-upcoming-projects.csv,COMP_DATE,road-ahead-current-road-closures.csv#COMP_DATE...
1,eo_pr.csv,NTEE_CD,eo_xx.csv,NTEE_CD,eo_pr.csv#NTEE_CD#eo_xx.csv#NTEE_CD
2,eo4.csv,NAME,eo_pr.csv,NAME,eo4.csv#NAME#eo_pr.csv#NAME
3,eo4.csv,ICO,eo_pr.csv,ICO,eo4.csv#ICO#eo_pr.csv#ICO
4,eo4.csv,STREET,eo_pr.csv,STREET,eo4.csv#STREET#eo_pr.csv#STREET


In [10]:
groundTruth.shape

(53, 5)

In [12]:
tp = 0
fp = 0
fn = 0

for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] in groundTruth.iloc[:,4].values):
        tp += 1
    else:
        fp += 1
    
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        fn += 1
    
precision = (tp)/(tp+fp)
recall = (tp)/(tp+fn)
f1_score = 2 * (precision*recall)/(precision+recall)

print(precision, recall, f1_score)

0.25396825396825395 0.9056603773584906 0.396694214876033


In [14]:
tp, fp, fn

(48, 141, 5)

# Falso Positivo

In [15]:
for i in range(len(cta_matches)):
    if(cta_matches.iloc[i,4] not in groundTruth.iloc[:,4].values):
        print(cta_matches.iloc[i,4])

eo_pr.csv#EIN#eo_xx.csv#EIN
eo4.csv#EIN#eo_xx.csv#EIN
eo_xx.csv#NAME#schools.csv#SCHOOL_NAME
eo_pr.csv#NAME#eo_xx.csv#NAME
eo_xx.csv#NAME#rental-standards-current-issues.csv#BUSINESSOPERATOR
community-gardens-and-food-trees.csv#STEWARD_OR_MANAGING_ORGANIZATION#eo_xx.csv#NAME
eo_pr.csv#ICO#eo_xx.csv#ICO
SCS_Staff_Salaries_data_30th_June 2010.csv#Name#eo_xx.csv#ICO
eo_xx.csv#STREET#schools.csv#ADDRESS
eo_pr.csv#STREET#eo_xx.csv#STREET
eo_xx.csv#STREET#rental-standards-current-issues.csv#Street
eo_xx.csv#STREET#libraries.csv#ADDRESS
eo_xx.csv#STREET#road-ahead-projects-under-construction.csv#STREET
eo_pr.csv#STATE#eo_xx.csv#STATE
eo_xx.csv#STATE#statewise-census-data-in-india-1901-2011.csv#LOCATION_NAME
eo_pr.csv#ZIP#eo_xx.csv#ZIP
eo4.csv#GROUP#eo_xx.csv#GROUP
eo4.csv#ORGANIZATION#eo_xx.csv#GROUP
eo_pr.csv#SUBSECTION#eo_xx.csv#SUBSECTION
eo_pr.csv#STATUS#eo_xx.csv#SUBSECTION
eo4.csv#SUBSECTION#eo_xx.csv#SUBSECTION
eo_pr.csv#SUBSECTION#eo_xx.csv#CLASSIFICATION
eo_pr.csv#STATUS#eo_xx.csv#CL

# Falso Negativo

In [13]:
for i in range(len(groundTruth)):
    if(groundTruth.iloc[i,4] not in cta_matches.iloc[:,4].values):
        print(groundTruth.iloc[i,4])

eo4.csv#ICO#eo_pr.csv#ICO
eo4.csv#CITY#eo_pr.csv#CITY
eo4.csv#SORT_NAME#eo_pr.csv#SORT_NAME
eo4.csv#ICO#eo_xx.csv#ICO
eo4.csv#SORT_NAME#eo_xx.csv#SORT_NAME
